In [16]:
import duckdb
import pandas as pd
import pyarrow.parquet as pq
from itertools import islice, tee

from src.core import Data, Reader

In [17]:
d = "/home/anadirov/Documents/big.parquet"
# pd_df = pd.read_parquet(d)

# q = f"""
#     create table data as select * from "{d}"
# """.strip()

In [18]:
df = duckdb.read_parquet(d)

In [4]:
parquet_file = pq.ParquetFile(d)
gen = parquet_file.iter_batches(batch_size=1000)

In [5]:
def nth_item(generator, n):
    return next(islice(generator, n, n+1), None)

In [6]:
def count_items(generator):
    return sum(1 for _ in generator)

def count_and_copy(generator):
    gen_copy, gen_count = tee(generator)
    count = sum(1 for _ in gen_count)
    return gen_copy, count


# count_and_copy(gen)

In [7]:
r = df.query("data", "select * from data limit 10000")

In [8]:
bats = r.to_arrow_table().to_batches(10)

In [9]:
big = Data(path = d, 
           virtual_table_name = "data", 
           query = "select item_id, __index_level_0__ from data order by __index_level_0__")

In [11]:
big.columns

['item_id', 'user_id', 'rank_ctb', '__index_level_0__']

In [12]:
r = big.search('700506', "item_id")

In [ ]:
# df.query("data",
#          "select item_id, __index_level_0__ from data order by __index_level_0__")\
#          .query("data_", 
#                 """SELECT * 
#                     FROM data
#                     WHERE CAST(item_id as VARCHAR) ILIKE '%700506%'
#                 """)